# Extração de Features e Construção do Modelo

### 1. Introdução a Extração de Features
- A extração de features é uma parte muito importante na análise e na identificação de relações entre diferentes elementos. Como já sabemos, os dados de áudio não podem ser compreendidos diretamente pelos modelos, então precisamos convertê-los para um formato inteligível, e é para isso que a extração de features é utilizada.


#### Com a taxa de amostragem e os dados do sinal, podemos realizar diversas transformações para extrair características valiosas do áudio. No entanto, neste projeto, não vamos aprofundar no processo de seleção de features para identificar quais são mais relevantes para o nosso dataset. Em vez disso, vamos extrair cinco features principais para treinar nosso modelo:
- **Zero Crossing Rate (ZCR):** Mede a taxa de mudanças de sinal no áudio, ou seja, quantas vezes ele cruza o eixo zero em um determinado intervalo de tempo. Essa feature é útil para distinguir sons percussivos e não percussivos.
- **Chroma STFT:** Representa a energia espectral em 12 bins correspondentes às notas da escala musical ocidental. Essa característica é útil para identificar padrões harmônicos no áudio.
- **MFCC (Mel-Frequency Cepstral Coefficients):** Converte a frequência do áudio para a escala mel, aproximando-se da percepção auditiva humana. É uma das features mais utilizadas em reconhecimento de fala e emoção.
- **RMS (Root Mean Square):** Mede a energia do sinal ao calcular a média quadrática das amplitudes do áudio. Essa feature ajuda a representar a intensidade do som.
- **Mel Spectrogram:** Representa a distribuição de energia do áudio em diferentes faixas de frequência na escala mel, capturando informações espectrais essenciais.

### 1.1. Importando as bibliotecas

In [ ]:
import pandas as pd
import numpy as np

import pickle
import os
import sys

import librosa
import librosa.display
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 

### 1.2. Carregar o Dataframe da EDA 

In [ ]:
data_path = "C:/Users/Vinícius/Desktop/miniprojeto Trilha/miniprojeto2/data/ravdess_preprocessed.csv" 
df = pd.read_csv(data_path)

In [ ]:
df.tail()

### 2. Criando uma função para extração das features

##### Fizemos essa parte no notebook passado, então você pode copiar e colar o código das funções aqui, pois precisaremos delas:

In [ ]:
def noise(data):
    n = np.random.normal(0, data.std(), data.size) # cria um vetor com ruído branco 
    return data + n # retorna o áudio com ruido

def stretch(data, rate=0.8):
    return librosa.effects.time_stretch(data, rate=rate) # retonra um áudio com velocidade alterada

def shift(data):
    return np.roll(data, np.random.random(1, len(data))) # retorna um vetor de audio com os dados deslocados aleatoriamente

def pitch(data, sampling_rate, pitch_factor=0.7):
    return librosa.effects.pitch_shift(y=data, sr=sampling_rate, n_steps=pitch_factor) # retorna um audio com a frequência alterada

# Tomando um áudio qualquer com exemplo
path = np.array(df.Path)[1]
data, sample_rate = librosa.load(path)

##### Aqui você deve extrair essas features de fato... mais tarde você precisará delas :)

# **Funções usadas**

### **vstack()**: concatena dois arrays numpy por linha e gera uma matriz

[1, 2, 3] e [4, 5, 6] -> [[1, 2, 3,]
                          [4, 5, 6,]]  

### **hstack()**: concatena pela coluna e gera uma matriz.

[1, 2, 3] e [4, 5, 6] -> [[1, 2, 3, 4, 5, 6]]  


### **zero.crossings()**: calcula quantos ZCR existem em um áudio.  
- Retorna um array numpy de **tamanho igual ao vetor de áudio**, em que cada elemento indica se **passou pelo eixo X ou não** 
- **Parâmetros**: (audio, pad=False), pan=False deixa de contar com o primeiro valor do array audio[0]  
  
### **chroma_stft()** retorna o espectograma de notas  
- **Parâmetros**: (audio, sample rate)
- Com o espectograma, é possível descobrir a tonalidade de uma música
- Para descobrir quantos existem em um áudio: utilizar a função sum  

 ### **np.mean()**: Para cada feature, é necessário reduzir sua dimensão, pois a função hstack só aceita arrays com o mesmo numero de linhas (). Para isso, tomamos a média dos valores de cada feature para jogar no ML


# **Escala mel**: escala que tenta simular numericamente a percepção humana do som
- Mudanças de sons em baixa frequência são mais fáceis de serem percebidos, mas mudança em sons de alta frequência quase não percebemos

### **.mfcc()**: retorna um áudio em escala mel para cada frame do áudio 
- **Parâmetros**: (audio, sr, n_mfcc)
 - **n_mfcc** é a quantidade de dados que iremos tomar sobre o audio na escala mel. Tomamos 13 porque já são suficientes para detectar padrões e é um bom número para treinar modelos de machine learning com desempenho  

### **.melspectogram()**: retorna um array com o espectograma do áudio em escala mel


In [ ]:
def extract_features(data, sample_rate): 
    result = np.array([])
    # zcr
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data, pad=False), axis=1)
    result = np.hstack((result, zcr))

    # Chroma_stft
    chroma_stft = np.mean(librosa.feature.chroma_stft(y=data, sr=sample_rate), axis=1)
    result = np.hstack((result, chroma_stft)) 

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate), axis=1) # n_mfcc = 20, pois com 20 amostras já é possível detectar padrões no áudio 
    result = np.hstack((result, mfcc))

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data, frame_length=2048, hop_length=512), axis=1)
    result = np.hstack((result, rms))

    # MelSpectrogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr = sample_rate), axis=1)
    result = np.hstack((result, mel))
    
    return result # retorna um array com 1x162, em que cada coluna é o dado de uma feaure 

def get_features(path):
    # Carregar áudio
    data, sample_rate = librosa.load(path, duration=2.5, offset=0.6)

    # Sem aumento de dados
    res1 = extract_features(data, sample_rate)  
    result = np.array(res1)

    # Com ruído
    noise_data = noise(data) # aplica áudio com ruído
    res2 = extract_features(noise_data, sample_rate)  
    result = np.vstack((result, res2))

    # Com alongamento e mudança de pitch
    new_data = stretch(data) # altera o tempo
    data_stretch_pitch = pitch(new_data, sample_rate) # altera a afinação (pitch) 
    res3 = extract_features(data_stretch_pitch, sample_rate) 
    result = np.vstack((result, res3)) # result é o empilhamento de linhas de todos as features
    
    return result # retorna um array 3x162, em que todas as linhas representam o mesmo áudio e as linhas são as features desse áudio 

##### Rode as células abaixo

## Ideia do código:  
   
1° Itera em cada linha do dataset capturando o path do áudio e a emoção associada.  

2° Chama a função que captura as features de um áudio
- A função retorna uma matriz 3x5, em que **a primeira linha é o áudio normal, a segunda é o áudio com ruído e a terceira é o áudio com alteração no pitch+stretch**
- As colunas são as 5 features associadas a cada áudio  

3° Adiciona cada linha, ou seja, cada **informação sobre a feature** na lista X

4° Adiciona a emoção associada a essa feature na lista Y


Assim, cada elemento de X se relaciona com sua respectiva emoção salva em Y. Exemplo:  

X = [ [1, 2, 3, 4, 5], [4, 3, 1, 6, 7], [7, 8, 6, 4, 1], [4, 3, 2, 1, 5], [12, 34, 66, 88, 11], [32, 14, 53, 67, 12] ]  

Y = [ 'fear', 'fear', 'fear', 'happy', 'happy', 'happy']

In [ ]:
# X salva as features do áudio
# Y salva as emoções associadas as features
X, Y = [], []

# zip retorna uma dupla em que cada elemento é uma dupla: um de path e outro de emotions
for path, emotion in zip(df['Path'], df['Emotions']):
    feature = get_features(path) # retorna um array 3x5 [ [1, 2, 3, 4, 5], [1, 2, 3, 4, 5], [1, 2, 3, 4, 5] ]

    for ele in feature:
        X.append(ele)
        Y.append(emotion)

In [ ]:
    # essa linha retorna uma tupla que informa o tamanho de X, Y, número de áudios
# como cada áudio sofre data augmentation 3x, então o tamanho de X  Y é 3 vezes maior que o número de áudios 

len(X), len(Y), df.Path.shape

##### Agora você deverá salvar o csv de features (features.csv)

## **Explicação do código**:

Após extrair X e Y, temos as informações de cada feature associada a uma única emoção. Agora, a ideia é criar um DataSet em que cada linha contenha as features de um áudio e sua respectiva emoção 

In [ ]:
# os.path.dirname() retorna o nome de um diretório
# os.getcwd() retorna o diretório pai 
data_dir = os.path.join(os.path.dirname(os.getcwd()), "data") # acessa o diretório pai na pasta 'data'
os.makedirs(data_dir, exist_ok=True)

# armazena a informação de onde será salvo o CSV criado
csv_path = os.path.join(data_dir, "features.csv")

# Crie um DataFrame do pandas chamado Features usando a lista X.
Features = pd.DataFrame(X)

# Transformando Y em um dataset com uma única coluna
labels = pd.DataFrame(Y, columns=['labels'])

# Concatenando o dataset labels a features
Features = pd.concat([Features, labels], axis=1)

# Salve o DataFrame como um arquivo CSV no caminho definido, sem incluir o índice, definindo _index_=False
Features.to_csv(csv_path, index=False)


print(f"O arquivo foi salvo em: {csv_path}")

In [ ]:
Features

### 2.1. Data Preparation (Preparação dos Dados)

# **Funções usadas:**  

### **.iloc()**: serve para selecionar linhas e colunas específicas de um dataframe. Exemplo:  
- .iloc[0:10, -1]: seleciona as linhas de 0 a 4 da última coluna. Usa-se a vírgula para separar linha/coluna 
- .iloc[0, :-1]: seleciona a primeira linha de todas as colunas menos a última  
- **Retorno**: **uma series pandas**


### **.get_dummies()**: serve para transformar um array numpy em dados categoricos
- **Parâmetros**: (array numpy, dtype=int)

In [ ]:
# seleciona todas as linhas da primeira até a penultima coluna
# aplica .values para que cada elemento de X seja uma linha do dataset 
X = Features.iloc[: ,:-1].values 

# Retorna todos os dados de labels em um array
Y = Features['labels'].values

In [ ]:
# Use OneHotEncoder para transformar o Y em uma representação binária categórica, necessária para problemas de classificação multiclasse
Y = pd.get_dummies(Y, dtype= int) 
Y

### **train_test_split**: divide cada matriz passada em duas: dados para teste e dados para treino  

- **Parâmetros**: (matriz1, matriz2, ..., train_size=, test_size=)
- Divide **ALEATORIAMENTE** quais dados usará no conjunto teste ou treino
- Mesmo passando duas matrizes, ela vai dividir as **mesmas linhas**


In [ ]:
# Use train_test_split do scikit-learn para dividir X e Y em conjuntos de treino e teste
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.15) # divide em 15% dos dados para teste e 85% para treino

X_train

### **StandardScaler**: é uma fórmula para normalizar os dados:

z = (X - média da coluna de X) / desvio padrão de X  

**.fit()**: calcula o novo valor para os dados  
  
**.transform()**: aplica a transformação de valores  

Em X_test não é necessário usar o .fit(), pois o calculo ja foi feito anteriormente, so basta aplicá-lo com transform

In [ ]:
# Utilize StandardScaler do sklearn para normalizar as características de X
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train.shape, X_test.shape

In [ ]:
# Rode essa célula para deixar as dimensões certinhas com o modelo que iremos criar.
X_train = np.expand_dims(X_train, axis=2)
X_test = np.expand_dims(X_test, axis=2)
X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

### 3. Model Training (Modelling)

#### O modelo que vamos usar é uma rede neural convolucional (CNN) projetada para processar os dados extraídos dos arquivos de áudio. Essa estrutura é ideal para capturar padrões espectrais, como variações de tom e intensidade. A CNN é composta por:
- Camadas Convolucionais (Conv1D): Extraem características do espectro do áudio.
- Camadas de Pooling (MaxPooling1D): Reduzem a dimensionalidade e capturam as informações mais relevantes.
- Dropout: Ajuda a evitar overfitting.
- Camada Flatten: Transforma os mapas de features em um vetor de entrada para a camada totalmente conectada.
- Camadas Densas (Dense): Realizam a classificação final usando a função de ativação softmax.

Uma imagem é uma grande matriz. Em que cada ponto é uma cor RGB. Assim, computar todos os pontos dessa matriz é algo extremamente custoso.  

A rede CNN **tenta extrair as características mais importantes em uma imagem**. Exemplo, ao se deparar com a imagem de ondas sonoras de um áudio, tenta computar somente as partes que as ondas aparecem  

# **Funções usadas:**  

### **keras.sequential()**: a rede CNN utilizada possui uma única entrada. A função serve para escrever camadas em Redes desse tipo  

### **keras.layers.Conv1D()**: cria uma camada de CNN para 1 dimensão  

Em uma rede CNN, é importante saber quais áreas da imagem são as mais importantes:  
### **keras.layers.MaxPooling1D** é a função que reduz a imagem na CNN  

## **keras.layers.Dropout**: elimina aleatoriamente algumas conexões da rede de CNN
- O objetivo é impedir o **Overffiting**, pois, ao eliminar conexões, diminui a probabilidade de o modelo estar excessivamente treinado com seus próprios dados 
- mais infos: https://medium.com/@csediveyanand/what-is-dropout-in-keras-deep-learning-c2d0913439f3

### **keras.Flatten**: ao entrar com imagens de diferentes dimensões, o Flatten as converte para um array unidimensional  
- Isso acontece para que o modelo só precise de um único neurônio de entrada
- mais infos: https://kevinmlean.medium.com/how-to-use-keras-layers-flatten-c3f29ed1b686

In [ ]:
# Dica: você pode olhar a documentação e ir seguindo o passo a passo arquitetônico para criar o modelo.

# Passo 1: Use Sequential() para criar o modelo como um contêiner linear.
# Passo 2: Adicione uma camada Conv1D com 256 filtros, kernel_size=5, strides=1, padding='same' e função de ativação 'relu'.
# Passo 3: Siga com uma camada MaxPooling1D com pool_size=5, strides=2, padding='same'.
# Passo 4: Adicione mais uma camada Conv1D com 256 filtros, kernel_size=5, strides=1, padding='same' e função de ativação 'relu'.
# Passo 5: Adicione mais uma camada MaxPooling1D com pool_size=5, strides=2, padding='same'.
# Passo 6: Adicione mais uma camada Conv1D com 128 filtros, kernel_size=5, strides=1, padding='same' e função de ativação 'relu'.
# Passo 7: Adicione mais uma camada MaxPooling1D com pool_size=5, strides=2, padding='same'.
# Passo 8: Adicione uma camada Dropout com 0.2 de taxa de dropout.
# Passo 9: Adicione mais uma camada Conv1D com 64 filtros, kernel_size=5, strides=1, padding='same' e função de ativação 'relu'.
# Passo 10: Adicione mais uma camada MaxPooling1D com pool_size=5, strides=2, padding='same'.
# Passo 11: Adicione uma camada Flatten.
# Passo 12: Adicione uma camada Dense com 32 unidades e função de ativação 'relu'.
# Passo 13: Adicione uma camada Dropout com 0.3 de taxa de dropout.
# Passo 14: Adicione uma camada Dense com 8 unidades e função de ativação 'softmax'.
# Passo 15: Compile o modelo com otimizador 'adam', loss 'categorical_crossentropy' e métrica 'accuracy'.
# Passo 16: Use model.summary() para visualizar o modelo.

modelo = keras.Sequential( # passo 1
    [
     keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding='same', activation='relu',input_shape=(162, 1)), #neurônio de entrada # passo 2
     keras.layers.MaxPooling1D(pool_size=5, strides=2, padding='same'), # passo 3
     keras.layers.Conv1D(filters=256, kernel_size=5, strides=1, padding='same', activation='relu'), # passo 4
     keras.layers.MaxPooling1D(pool_size=5, strides=2, padding='same'), # passo 5
     keras.layers.Conv1D(filters=128, kernel_size=5, strides=1, padding='same', activation='relu'), # passo 6
     keras.layers.MaxPooling1D(pool_size=5, strides=2, padding='same'), # passo 7
     keras.layers.Dropout(rate=0.2), # passo 8
     keras.layers.Conv1D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu'), # passo 9
     keras.layers.MaxPooling1D(pool_size=5, strides=2, padding='same'), # passo 10
     keras.layers.Flatten(), # passo 11
     keras.layers.Dense(units=32,activation='relu'), # passo 12
     keras.layers.Dropout(rate=0.3), # passo 13
     keras.layers.Dense(units=8,activation='softmax')
                                                    ]
)

# adam é um otimizador para atualizar os pesos durante o treino
# loss é a função de perda, ou seja, o que o modelo deve minizar (diferença entre o previsto e o real)
modelo.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) #passo 15
modelo.summary() #passo 16

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_14 (Conv1D)              │ (None, 162, 256)       │         1,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_12 (MaxPooling1D) │ (None, 81, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_15 (Conv1D)              │ (None, 81, 256)        │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_13 (MaxPooling1D) │ (None, 41, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_16 (Conv1D)              │ (None, 41, 128)        │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_14 (MaxPooling1D) │ (None, 21, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 21, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_17 (Conv1D)              │ (None, 21, 64)         │        41,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_15 (MaxPooling1D) │ (None, 11, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 704)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │        22,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 8)              │           264 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 557,288 (2.13 MB)

 Trainable params: 557,288 (2.13 MB)

 Non-trainable params: 0 (0.00 B)

### 3.1. Agora vamos de fato treinar o modelo, siga esses passos: 
##### 1. Use o callback para Ajustar a Taxa de Aprendizado 
##### ReduceLROnPlateau: Diminui a taxa de aprendizado quando uma métrica está estagnada.
   Parâmetros:
   - monitor='loss': Monitora a perda durante o treinamento.
   - factor=0.4: Reduz a taxa de aprendizado por este fator.
   - patience=2: Número de épocas sem melhora antes da redução.
   - min_lr=0.0000001: Limite mínimo para a taxa de aprendizado.
             
##### 2. Treine o Modelo 
##### Utilize model.fit para iniciar o treino da rede neural.
   Parâmetros:
   - x_train, y_train: Conjunto de dados de treino.
   - batch_size=64: Número de amostras por atualização de gradiente.
   - epochs=50: Número de vezes que o modelo treina em todo o conjunto de dados.
   - validation_data=(x_test, y_test): Conjunto de dados para validação durante o treino.
   - callbacks=[rlrp]: Lista de callbacks a serem aplicados durante o treino.
     

In [223]:
modelo.fit(
    X_train, Y_train, batch_size=64, epochs=50, validation_data= (X_test,Y_test),
    callbacks=[ReduceLROnPlateau(monitor= 'loss', factor= 0.4, patience= 2, min_lr= 0.0000001)])

Epoch 1/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - accuracy: 0.2007 - loss: 2.0084 - val_accuracy: 0.2454 - val_loss: 1.9604 - learning_rate: 0.0010
Epoch 2/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.2321 - loss: 1.9322 - val_accuracy: 0.2562 - val_loss: 1.8893 - learning_rate: 0.0010
Epoch 3/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 58ms/step - accuracy: 0.2761 - loss: 1.8694 - val_accuracy: 0.3117 - val_loss: 1.8034 - learning_rate: 0.0010
Epoch 4/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - accuracy: 0.2948 - loss: 1.8150 - val_accuracy: 0.3025 - val_loss: 1.7817 - learning_rate: 0.0010
Epoch 5/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step - accuracy: 0.3046 - loss: 1.7771 - val_accuracy: 0.3102 - val_loss: 1.7516 - learning_rate: 0.0010
Epoch 6/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.3182 - loss: 1.7503 - val_accuracy: 0.3210 - val_loss: 1.7280 - learning_rate: 0.0010
Epoch 7/50
58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 55ms/step - accuracy: 0.3442 - loss: 1.6977 - val_acc

### 3.2. Avaliar e Visualizar a Performance do Modelo

##### Passo 1: Avaliando o Modelo
- **Precisão nos Dados de Teste**:
  - Use `model.evaluate(x_test, y_test)` para calcular a precisão do modelo no conjunto de teste.

##### Passo 2: Preparando os Gráficos
- **Definindo Épocas**:
  - Crie uma lista de épocas para o eixo x
- **Configurando o Layout do Gráfico**:
  - Use `fig, ax = plt.subplots(1, 2)` para criar dois gráficos lado a lado.

##### Passo 3: Plotando a Perda
- **Gráfico de Perda**:
  - Plote a perda de treino e teste:
    ```python
    ax[0].plot(epochs, train_loss, label='Training Loss')
    ax[0].plot(epochs, test_loss, label='Testing Loss')
    ```

##### Passo 4: Plotando a Precisão
- **Gráfico de Precisão**:
  - Plote a precisão de treino e teste:
    ```python
    ax[1].plot(epochs, train_acc, label='Training Accuracy')
    ax[1].plot(epochs, test_acc, label='Testing Accuracy')
    ```

##### Objetivos:
  - **Perda**: Ajuda a identificar se o modelo está treinando bem ou se há overfitting.
  - **Precisão**: Mostra o quão eficaz é o treinamento do modelo em acertar as previsões.


In [ ]:
# Code here

### 3.3. Chegou a parte boa! vamos prever os valores nos dados de teste
- Passo 1: Use model.predict() no x_test e salve o resultado em pred_test.
- Passo 2: Crie y_pred a partir de pred_test usando inverse_transform do encoder (Precisamos converter as previsões codificadas do One-Hot de volta aos rótulos originais)
- Passo 3: Faça o mesmo para os rótulos de teste (y_test), decodificando-os de volta aos rótulos originais.

In [ ]:
# Code here

In [ ]:
# Rode essa célula para ver se o modelo que criamos está fazendo sentido para a maioria dos valores.

df = pd.DataFrame(columns=['Predicted Labels', 'Actual Labels'])
df['Predicted Labels'] = y_pred.flatten()
df['Actual Labels'] = y_test.flatten()

df.head(10)

### 3.4. Plotar Matriz de Confusão
- Agora tenho um desafio para você, eu quero que você crie uma matriz de confusão que relacione os resultados preditos com os valores reais das emoções!

In [ ]:
# Code here

In [ ]:
# Use a função classification_report do sklearn para visualizar a precisão, recall e f1-score do modelo.
# Code here

### 4. Conclusão
- Podemos ver que nosso modelo é mais preciso na predição das emoções surpresa e raiva, o que faz sentido, pois os arquivos de áudio dessas emoções diferem bastante dos outros em aspectos como tom, velocidade, etc.
- No geral, alcançamos 62% de precisão nos dados de teste, o que é razoável, mas podemos melhorar ainda mais aplicando mais técnicas de aumento de dados e utilizando outros métodos de extração de features.

### 5. Salvando o Modelo e o Scaler

##### Passo 1: Salvando o Modelo
1. **Importação:** Use o `load_model` do Keras.
2. **Diretório:** Crie um diretório chamado `models` se não existir.
3. **Salvar:** Salve o modelo como no caminho especificado.

##### Passo 2: Salvando o Scaler
1. **Importação:** Use `joblib`.
2. **Diretório:** Utilize o mesmo caminho `models`.
3. **Salvar:** Salve o scaler.

In [ ]:
# Code here

In [ ]:
# Code here